In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
import tensorflow as tf
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(300, 300, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# the model so far outputs 3D feature maps (height, width, features)

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))
# COMPILE
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
model.summary()

Using TensorFlow backend.







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 298, 298, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 298, 298, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 149, 149, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 147, 147, 32)      9248      
_________________________________________________________________
activation_2 (Activation)    (None, 147, 147, 32)      0         
____________________________________

In [ ]:
batch_size = 16

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        zoom_range=0.2,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        '/content/drive/My Drive/Project/Data/CNNData/Train',  # this is the target directory
        target_size=(300, 300),  # all images will be resized to 300x300
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        '/content/drive/My Drive/Project/Data/CNNData/Test',
        target_size=(300, 300),
        batch_size=batch_size,
        class_mode='binary')

save_callback = ModelCheckpoint('CNN_Models_ShapeDetection_save_best_epoch.h5', monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='min', period=10)

class CustomModelCheckpoint(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        # logs is a dictionary
        if epoch % 10 == 0:
          #print(f"epoch: {epoch}, train_acc: {logs['acc']}, valid_acc: {logs['val_acc']}")
          #if logs['val_acc'] > logs['acc']: # your custom condition
          self.model.save('CNN_Models_ShapeDetection_' + str(epoch) + '_model.h5')

save_per_epoch = CustomModelCheckpoint()

history = model.fit_generator(
        train_generator,
        steps_per_epoch=225,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=25,
        callbacks = [save_callback, save_per_epoch])

Found 3599 images belonging to 2 classes.
Found 400 images belonging to 2 classes.



Epoch 1/50





225/225 [==============================] - 1406s 6s/step - loss: 0.4789 - acc: 0.8288 - val_loss: 0.4524 - val_acc: 0.8225
Epoch 2/50
225/225 [==============================] - 90s 402ms/step - loss: 0.4228 - acc: 0.8316 - val_loss: 0.3573 - val_acc: 0.8225
Epoch 3/50
225/225 [==============================] - 95s 422ms/step - loss: 0.3557 - acc: 0.8330 - val_loss: 0.2855 - val_acc: 0.8225
Epoch 4/50
225/225 [==============================] - 95s 422ms/step - loss: 0.3124 - acc: 0.8503 - val_loss: 0.2530 - val_acc: 0.8325
Epoch 5/50
225/225 [==============================] - 95s 420ms/step - loss: 0.2907 - acc: 0.8667 - val_loss: 0.2090 - val_acc: 0.9375
Epoch 6/50
225/225 [==============================] - 94s 417ms/step - loss: 0.2659 - acc: 0.8753 - val_loss: 0.2117 - val_acc: 0.9200
Epoch 7/50
225/225 [==============================] - 94s 418ms/step - loss: 0.2591 - acc: 0.8871 - 

In [ ]:
acc = acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

NameError: ignored